In [1]:
from sklearn.ensemble import GradientBoostingClassifier
import pickle
from sklearn.metrics import ndcg_score
import pandas as pd
import gc

In [2]:
# training set
x = pd.read_csv("data/dummy/train.csv")

# val set 20%
x_val = x.sample(n=int(0.1*len(x)))
x = x.drop(x_val.index)

## TODO: date_time
x.drop(['click_bool', 'booking_bool', 'date_time'], axis=1, inplace=True)
x_val.drop(['click_bool', 'booking_bool', 'date_time'], axis=1, inplace=True)

# sort both on search id
x.sort_values('srch_id', ignore_index=True)
x_val.sort_values('srch_id', ignore_index=True)

,srch_id,site_id,visitor_location_country_id,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,...,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,orig_destination_distance,random_bool,agg_comp_rate,agg_comp_perc,agg_comp_inv,target
0,4,5,219,219,37581,5,4.5,0,3.09,0.2508,...,1,0,1,1,238.35,1,0.0,0.0,0.0,0
1,4,5,219,219,11826,5,4.5,1,3.22,0.1924,...,1,0,1,1,238.33,1,0.0,0.0,0.0,0
2,12,5,219,158,7814,3,3.5,0,0.00,0.0476,...,3,0,1,1,9270.55,1,0.0,0.0,0.0,0
3,12,5,219,158,10881,3,3.0,0,3.14,0.0000,...,3,0,1,1,9285.88,1,1.0,16.0,0.0,0
4,17,18,129,31,87596,3,2.5,0,6.91,0.7270,...,4,1,2,1,433.87,1,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121569,332765,32,220,98,8507,4,4.5,1,4.14,0.3116,...,2,0,1,0,4484.67,1,0.0,0.0,0.0,0
121570,332765,32,220,98,17709,4,0.0,0,3.33,0.0000,...,2,0,1,0,4483.77,1,0.0,0.0,0.0,0
121571,332768,5,219,219,13618,2,4.5,0,1.39,0.2227,...,4,0,1,1,104.68,0,-1.0,11.0,0.0,0
121572,332777,16,31,219,10876,3,3.5,1,3.18,0.0466,...,2,1,1,0,4940.48,1,0.0,0.0,0.5,0


In [3]:
y_val = x_val.target.squeeze()
y = x.target.squeeze()
x.drop(['target'], axis=1, inplace=True)
x_val.drop(['target'], axis=1, inplace=True)

In [4]:
groups = x.srch_id.value_counts().sort_index()
groups_val = x_val.srch_id.value_counts().sort_index()

In [5]:
import lightgbm

# default lightgbm model with sklearn api
gbm = lightgbm.LGBMRanker(metric="ndcg", objective="lambdarank") 

# fitting model 
gbm.fit(
    x,
    y,
    eval_set = [(x_val, y_val)],
    group = groups,
    eval_group = [groups_val],
    verbose=1,
    early_stopping_rounds=100
)

[1]	valid_0's ndcg@1: 0.986222	valid_0's ndcg@2: 0.994147	valid_0's ndcg@3: 0.994745	valid_0's ndcg@4: 0.994775	valid_0's ndcg@5: 0.994775
Training until validation scores don't improve for 100 rounds
[2]	valid_0's ndcg@1: 0.986751	valid_0's ndcg@2: 0.994392	valid_0's ndcg@3: 0.994955	valid_0's ndcg@4: 0.994981	valid_0's ndcg@5: 0.994981
[3]	valid_0's ndcg@1: 0.986909	valid_0's ndcg@2: 0.99444	valid_0's ndcg@3: 0.995009	valid_0's ndcg@4: 0.995035	valid_0's ndcg@5: 0.995035
[4]	valid_0's ndcg@1: 0.986857	valid_0's ndcg@2: 0.994464	valid_0's ndcg@3: 0.994994	valid_0's ndcg@4: 0.995025	valid_0's ndcg@5: 0.995025
[5]	valid_0's ndcg@1: 0.986898	valid_0's ndcg@2: 0.994435	valid_0's ndcg@3: 0.995006	valid_0's ndcg@4: 0.995032	valid_0's ndcg@5: 0.995032
[6]	valid_0's ndcg@1: 0.98691	valid_0's ndcg@2: 0.99446	valid_0's ndcg@3: 0.995009	valid_0's ndcg@4: 0.99504	valid_0's ndcg@5: 0.99504
[7]	valid_0's ndcg@1: 0.986878	valid_0's ndcg@2: 0.994435	valid_0's ndcg@3: 0.994995	valid_0's ndcg@4: 0.9950

[61]	valid_0's ndcg@1: 0.987577	valid_0's ndcg@2: 0.994677	valid_0's ndcg@3: 0.995265	valid_0's ndcg@4: 0.995283	valid_0's ndcg@5: 0.995283
[62]	valid_0's ndcg@1: 0.987609	valid_0's ndcg@2: 0.994689	valid_0's ndcg@3: 0.995276	valid_0's ndcg@4: 0.995295	valid_0's ndcg@5: 0.995295
[63]	valid_0's ndcg@1: 0.98762	valid_0's ndcg@2: 0.994689	valid_0's ndcg@3: 0.995279	valid_0's ndcg@4: 0.995298	valid_0's ndcg@5: 0.995298
[64]	valid_0's ndcg@1: 0.987599	valid_0's ndcg@2: 0.994694	valid_0's ndcg@3: 0.995274	valid_0's ndcg@4: 0.995293	valid_0's ndcg@5: 0.995293
[65]	valid_0's ndcg@1: 0.987609	valid_0's ndcg@2: 0.994691	valid_0's ndcg@3: 0.995277	valid_0's ndcg@4: 0.995296	valid_0's ndcg@5: 0.995296
[66]	valid_0's ndcg@1: 0.987588	valid_0's ndcg@2: 0.99469	valid_0's ndcg@3: 0.995271	valid_0's ndcg@4: 0.995289	valid_0's ndcg@5: 0.995289
[67]	valid_0's ndcg@1: 0.987556	valid_0's ndcg@2: 0.994665	valid_0's ndcg@3: 0.995256	valid_0's ndcg@4: 0.995275	valid_0's ndcg@5: 0.995275
[68]	valid_0's ndcg@1:

LGBMRanker(metric='ndcg', objective='lambdarank')

In [6]:
gc.collect()
with open('model/model', 'wb') as f:
    pickle.dump(gbm, f)

In [7]:
with open('model/model', 'rb') as f:
    model = pickle.load(f)

In [8]:
test = pd.read_csv("data/dummy/test.csv", index_col=False)
test.drop('date_time', axis=1, inplace=True)

In [9]:
predictions = model.predict(test)

In [10]:
len(predictions)

4959183

In [11]:
submission = test[["srch_id", "prop_id"]]
submission['pred'] = predictions

<ipython-input-11-f49d9f7448f2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission['pred'] = predictions


In [12]:
del test
gc.collect()

84

In [13]:
submission = submission.sort_values(['srch_id', 'pred'], ascending=[True, False])

In [14]:
submission.head(n=20)

,srch_id,prop_id,pred
23,1,99484,3.415941
24,1,123675,3.047525
25,1,128085,3.032698
22,1,95031,2.847980
28,1,139162,2.766827
26,1,128871,2.741150
20,1,90385,2.737973
27,1,134992,2.592952
18,1,82231,2.529708
12,1,61934,2.409493


In [15]:
submission[['srch_id', 'prop_id']].to_csv('submission.csv', index=False)